In [25]:
# imports

from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os
import json

In [84]:
#todo add preprossesing stage that selects the relevent parts of the job description
#todo rank keywords
#todo sort keywords
#todo fintune a model for writing a good bio
#todo fintune a model for clasification of keywords
#todo fintune a model for ranking of keywords
#todo add few shot example to keyword extractor

# constants
DESCRIPTION = 'description'
KEYWORDS = 'keywords'
JOB_TYPE = 'job_type'
BIO = 'bio'


marketeam_description = """
Role Description

As an AI Engineer, you will be responsible for tasks such as:
Crawling the web.
Creating synthetic Datasets.
Build AI agents (without langGraph).
Connect to external APIs. Create internal APIs.
FineTune Large Language Models for specific tasks.
...
This role is located in Tel Aviv, full-time, hybrid (3 days at the office), next to the train station (Rokach Blvd 97).

Qualifications

Experience with HugginFace, Torch, NLP, and LLMs.
High proficiency in Python. Advantage: AWS, GO, Ruby, Elixir/Erland.
Ability to work independently and collaboratively in a team.
Strong communication and interpersonal skills.
Git master.
"""

marke_keywords = "AI Engineer, Crawling, Synthetic Datasets, AI agents, Connect to external APIs, Create internal APIs, FineTune Large Language Models, HugginFace, Torch, NLP, LLMs, Python, AWS, GO, Ruby, Elixir, Erlang, Git, Communication, Interpersonal skills, Independent work, Team collaboration"

job_keywords_pmt = f"""You are an expert and experienced programmer with experience in programing, backend, data science, hr, team building and resume enhancing.i need yor help to analyze job descriptions and extract the most important keywords. Your task is to read the job description provided and identify the key skills, qualifications, requirments and responsibilities required for the role. Please output these keywords in a concise and clear list. Remember to focus on the specific skills and experiences mentioned in the job description, rather than general job duties. use ',' as a separator for the list of keywords. for example : {marketeam_description}
 keywords: nlp, Ai agents, LLM APIs, Finetuning, HugginFace, Torch, LLMs, Git, AWS, GO, Ruby, Elixir, Erlang, Python"""

org_bio = """Highly motivated individual with a keen interest in Machine Learning. Equipped with solid communication skills and a pas-
sion for problem-solving, I’m eager to contribute to a high-performing team in a fast-paced, technology-driven environment.
Dedicated to continuing leveraging my knowledge and skill-set by providing impactful solutions"
"""
BIO_REWRITING_SYS_PMT = f"""You are an expert in creating and editing cvs and resumes from your background in programming, nlp and data science i will give you a job description followed by keywords from that job description followed by a bio from my resume and i want you to adjust the bio according to the job description and keywords. only slightly change the bio and do not exceed the original bio length, and dont talk about experience talk about motivation and intrests  for example: job decription: {marketeam_description} , keywords: {marke_keywords} ,bio: {org_bio}. adjusted bio: 'Highly motivated individual with a keen interest in Machine Learning and NLP in particular, with experience in agentic
workflows. Equipped with solid communication skills and a passion for problem-solving. I’m eager to contribute to a high-
performing team in a fast-paced, technology-driven environment. Dedicated to continuing leveraging my knowledge and
skill-set by providing impactful solutions.'"""


BIO_REWRITING_SYS_PMT_v2 = f"""You are an expert in creating and editing cvs and resumes from your background in programming, nlp and data science i will give you a job description followed by a bio from my resume and i want you to adjust the bio according to the job description. only slightly change the bio and do not exceed the original bio length, and dont talk about experience talk about motivation and intrests  for example: job decription: {marketeam_description}  ,bio: {org_bio}. adjusted bio: 'Highly motivated individual with a keen interest in Machine Learning and NLP in particular, with experience in agentic
workflows. Equipped with solid communication skills and a passion for problem-solving. I’m eager to contribute to a high-
performing team in a fast-paced, technology-driven environment. Dedicated to continuing leveraging my knowledge and
skill-set by providing impactful solutions.'"""


In [76]:
all_skills = [] # fill


load_dotenv()
open_ai_key = os.getenv("OPEN_AI_API_KEY")
# Initialize the OpenAI client
client = OpenAI(api_key=open_ai_key)

job_db = read_json('job_db.json')
gpt3t="gpt-3.5-turbo"
gpt4o="gpt-4o"
gpt4t="gpt-4-turbo"
def ask_gpt(system_pmt,  user_pmt, model=gpt3t):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_pmt},\
            {"role": "user", "content": user_pmt},\
        ],
    )
    return completion.choices[0].message.content

In [85]:
# Functions
def write_json(descriptions):
    # write json
    with open('job_db.json', 'w') as f:
        json.dump(descriptions, f)


def read_json(json_file='job_db.json'):
    # read json
    with open(json_file, 'r') as f:
        des = json.load(f)
    return des

def parse_keyword_response(response):
    kwy_ind = response[:10].lower().find('keywords:')
    if kwy_ind != -1:
        response = response[kwy_ind+9:].strip()
    if response.find('\n') != -1:
        response = response.split('\n')
    else: response = response.split(',')
    keywords = [keyword.replace("- ", "").strip() for keyword in response]
    return keywords

def get_job_keywords(descirption, model=gpt3t):
    response = client.chat.completions.create(
        model=model,
        max_tokens=200,
        temperature=0.0,
        messages=[
            {"role": "system", "content": job_keywords_pmt},
            {"role": "user", "content": f"Extract keywords from the following job description: {descirption}"},\
        ],
    )
    return response.choices[0].message.content.strip()

def add_job(job_name, job_description, job_type):
    s = get_job_keywords(job_description, gpt4t) #using gpt4t
    keywords = parse_keyword_response(s)
    # bio = ask_gpt(BIO_REWRITING_SYS_PMT, f"Job decription: {job_description}, keywords: {keywords}, resume bio: {org_bio}", gpt4o)
    bio = ask_gpt(BIO_REWRITING_SYS_PMT_v2, f"Job decription: {job_description}, resume bio: {org_bio}", gpt4o)
    job_db[job_name] = {DESCRIPTION: job_description, KEYWORDS:keywords, JOB_TYPE:job_type, BIO:bio}
    write_json(job_db)
    print(bio)
    print(keywords)
    print(s)

def edit_bio(job_name, new_bio):
    job_db[job_name][BIO] = new_bio
    write_json(job_db)


In [86]:
job_des = """
About Us:

Insait IO is reshaping the banking, insurance, and fintech scenes with advanced predictive technology and proactive LLMs. We focus on crafting smart, data-driven strategies for maximum engagement. Our approach includes real-time analytics that redefine marketing, and a sales bot that simplifies the financial product selection process, offering a seamless and automated experience for everything from loans to insurance.


Role Overview:

We are seeking a motivated and talented intern with a background in Data Science (DS) and a strong proficiency in Python, NumPy, and Pandas. The ideal candidate will have experience with Large Language Models (LLMs), including concepts such as transformers and training, and familiarity with OpenAI, Claude, and Gemini APIs. Additionally, a significant advantage will be given to candidates with backend development experience using Flask, demonstrating the ability to build, deploy, and maintain web applications. This role offers an excellent opportunity to work on cutting-edge projects, integrating data science with robust backend solutions.


Key Responsibilities:

Develop and optimize LLMs
Build LLM-based analytics
Integrate and utilize APIs from OpenAI, Claude, and Gemini
Connect user-LLM interaction interfaces and models


Requirements:

Strong experience in Data Science with Python, NumPy, and Pandas
Knowledge of LLM concepts, including transformers and training techniques
Familiarity with OpenAI, Claude, and Gemini APIs
Excellent problem-solving skills and a proactive attitude
Strong communication and teamwork abilities


What We Offer:

Hands-on experience with advanced AI technologies
Mentorship from experienced AI professionals
Opportunity to contribute to impactful projects
"""
add_job('insait', job_des, 'ML')
# edit_bio('fiverr', "")


Adjusted bio: Highly motivated individual with a keen interest in Data Science and Large Language Models. Equipped with solid communication skills and a passion for problem-solving, I’m eager to contribute to a high-performing team in a fast-paced, technology-driven environment. Dedicated to continuing leveraging my knowledge and skill-set by providing impactful solutions in the fintech and banking sectors.
['Data Science', 'Python', 'NumPy', 'Pandas', 'Large Language Models', 'LLM', 'transformers', 'training', 'OpenAI', 'Claude', 'Gemini APIs', 'Flask', 'backend development', 'web applications', 'optimize LLMs', 'LLM-based analytics', 'user-LLM interaction', 'problem-solving skills', 'communication skills', 'teamwork skills']
Data Science, Python, NumPy, Pandas, Large Language Models, LLM, transformers, training, OpenAI, Claude, Gemini APIs, Flask, backend development, web applications, optimize LLMs, LLM-based analytics, user-LLM interaction, problem-solving skills, communication ski

In [71]:
edit_bio('fiverr', """Highly motivated Computer Science graduate with a keen interest in Software Engineering and Backend Devel-
opment in particular. Equipped with solid communication skills and a passion for problem-solving. I’m eager to
contribute to a high-performing R&D team working on huge-scale web products in a fast-paced, technology-driven
environment. Dedicated to continuing leveraging my knowledge and skill-set by providing impactful solutions.""")
write_json(job_db)

In [7]:
tymely = """
Tymely is the first startup to offer an AI Agent for eCommerce customer service that is indistinguishable human agents. Tymely’s agent handles retailers’ most complex customer service tickets end-to-end with unprecedented service quality, including exchanges of partial orders, changing shipping addresses, refunds, etc.As a backend developer on our team, you will be responsible for the server-side of our product. Ultimately, you should be able to develop and maintain functional and stable pipelines and web applications.
Proven work experience as a Back-end developer
3+ years of Python experience
Experience in building production-grade, cloud-native software (AWS preferred) with hands-on experience in cloud automation
Serverless experience preferred
Passionate about writing clean, maintainable code that will be used and modified by others, and able to use and modify other developers’ work rather than recreate it
Responsibilities
Participate in the entire application lifecycle, focusing on coding and debugging
Write clean code that is easy to share with other developers, maintain, and test
Take a major part in defining aspects of development from appropriate technology and workflow to coding standards
Liaise with developers, designers, and customers to identify new features
Follow emerging technologies
"""



marketeam_description = """
Role Description

As an AI Engineer, you will be responsible for tasks such as:
Crawling the web.
Creating synthetic Datasets.
Build AI agents (without langGraph).
Connect to external APIs. Create internal APIs.
FineTune Large Language Models for specific tasks.
...
This role is located in Tel Aviv, full-time, hybrid (3 days at the office), next to the train station (Rokach Blvd 97).

Qualifications

Experience with HugginFace, Torch, NLP, and LLMs.
High proficiency in Python. Advantage: AWS, GO, Ruby, Elixir/Erland.
Ability to work independently and collaboratively in a team.
Strong communication and interpersonal skills.
Git master.
"""


evona_description = """
They are looking for a Jr Data Scientist to join the team and help towards the development of industry-leading analytics software. This is a position for someone who loves working with data and can help process and analyze large amounts of unorganized data to drive analytical value for clients.


What you’ll do:



Under the direction of senior staff, design and implement probabilistic and deterministic algorithms for information extraction and fusion from a variety of remote sensing and proprietary data sources.
Perform exploratory data analysis for problem definition and refinement. Develop and test candidate algorithms to demonstrate tradeoffs for final selection.
Assemble and clean training and test datasets. Perform model training. Iterate to meet model performance requirements.
Develop and maintain Python-based applications for processing volatile live data streams on Google CloudPlatform.
Work closely with relational databases and execute complex queries to extract actionable insights from geospatial data.
Optimize application performance and scalability to ensure flawless operation under heavy load conditions.


What they’re looking for:



MS or BS degree in Computer Science (preferred), or STEM field.
Ability to apply machine learning, statistics, and optimization principles to problem solving.
Ability to build Cloud Platform highly scalable data processing pipelines for volatile live data streams.
Knowledge of geospatial data processing tools and Python packages is desirable
Knowledge of image processing, algorithms and machine learning libraries is desirable
The ability to work effectively in a team environment with remotely located staff in different time zones, demonstrating strong communication and coordination skills.

"""


checkpoint_description = """
Cloud Research and Development Team is looking for a curious and highly motivated and talented Cloud software developer who will design, plan, and develop the new unified hub of the group’s tools.


As part of the team, you will lead and participate in developing state of the art, high scale security hub for cloud environments for the entire CloudGuard organization.


Key Responsibilities

Backend and frontend development of essential, daily used, fast-growing tool’s hub.
Investigate, implement, and share new technologies and ideas.
Split the hub between internal and external services.
Design, implement and scale our existing components. You will work closely with your fellow tech leaders.
Design, build, and implement new tools in a verity of areas.
Working with the users, ensuring their needs are met with promptness and high-quality service.


Qualifications

B.Sc. in Computer Science from a leading university, and GPA over 80 – MUST
1+ years of software development experience in one or more general server-side languages (Python is mandatory)
Solid Computer Science skills in object-oriented design, data structures, algorithm design, problem solving, and complexity analysis
Expertise in computer networking, troubleshooting and specifically in public cloud.
Familiarity with AWS – advantage
Fast learner, learning new code and new technologies very quickly
Extremely high motivation with a deep desire to achieve outstanding achievements.
Ability to work under pressure
Excellent time management and interpersonal skills
Team player
"""

yotpo_description = """
You will Build data pipelines & large scale solutions based on ML models.
Work with big data technologies (e.g., AWS, spark, airflow, k8s), ML technologies (e.g., pytorch, keras, mlflow) and software engineering technologies (e.g., git, docker, jenkins).
Collaborate with highly experienced developers and researchers.
Constantly learn new stuff :)

Must have:

B.Sc. / M.Sc. in Computer Science.
0-2 years of experience in software development – preferably as an ML engineer, big data developer or backend developer.
At least 1 year of experience developing in python – including academic experience such as a final project, advanced courses, student positions etc.
Outstanding achievements in fundamental computer science courses – Data Structures, Algorithms, Systems Programming, Object Oriented Programming.
Fast learner.

Significant Advantages:

Experience working with a cloud technology stack (e.g., AWS, Azure).
Academic background or professional experience in big data development (e.g., databases, distributed systems).
Significant project in machine learning or big data development.
Academic background in Machine Learning or related topics.
Experience with Spark or SQL.
"""

wix_description = """ Job Description
Building Wix’s 3000 microservices and 32M lines of code requires high-scale CI/CD. Bazel is our build tool of choice and we front its Scala integration and work on building additional sophisticated features, world-class pipelines, and highly scalable services to support them. We do hardcore backend engineering and specialize in Bazel, its tooling, and integrations.
We also work closely with the developers at Wix to improve their day-to-day experience and eliminate velocity killers, from their IDE to GA.

As an engineer in the Bazel team, you’ll:

Build and maintain tooling that helps our developers build and deliver code faster

Build, and maintain highly scalable backend services providing unique CI/CD experiences to support Wix’s backend code.

Work as a part of the Bazel OSS community, participate in discussions and workgroups and help shape Bazel itself
Qualifications:
4+ years of experience in server-side technologies

In-depth understanding of Object-Oriented Programming (OOP) and SOLID principles

Skilled in debugging and investigating production issues and you’re also able to recognize and address bad code

You have a good knowledge and interest in microservice architecture

You’re responsible and can work independently

You actively keep up with industry trends and developments and you’re continuously searching for better solutions

Quick grasp of large and complex systems and their inner workings

Familiar with software engineering best practices (testing, code reviews, immutability, scalability, etc.)

Knowledgeable about bad practices and anti-patterns

Team player committed to the success of the team

An advantage if you:

Have experience using Bazel

Are familiar with Scala

Have worked with Docker and Kubernetes

Have a passion for DevEx-related topics: developer tooling, development environments, and build tools

About the team
We are Wix’s Infrastructure Developer Experience team. We have a suite of developer productivity tools that serve our 2,000 engineers. We believe that we can better support Wix engineers by connecting our products across key engineering workflows to boost efficiency. Our main focus is currently DevPortal, the front-end product of all our CI/CD offerings. This pluggable platform will allow all of our engineering teams to build and customize whatever they need to improve their day-to-day development experience. As a group, we generate 8K builds and 600 deployments per day.
"""

bluewhite_description = """
Bluewhite drives the growth of resilient farms through pioneering technology and a friendly user experience. Our vision is to bring sustainable innovation to growers worldwide. Bluewhite aims to build resilient farms by maximizing the grower's productivity and reducing operational costs. We equip existing fleets with autonomous technology, an easy-to-use platform, and end-to-end service, supporting their journey to a more profitable and sustainable farm. Together, we will Grow Forward.

Are you looking to be a part of a team that impacts the world's future daily? Our team does that every single day. We are a global company located in TLV, Israel, and California, USA, driving the growth of resilient farms through pioneering technology and a friendly user experience.

We are seeking a talented and experienced Data and ML Engineer, you’ll be leading the ML infrastructure and data processing R&D effort, taking a pivotal part in developing our proprietary machine learning-based perception and navigation algorithms.

In this position, you will report to the AI Team Lead and play a key role in a fast-growing team with significant impact and potential for professional and personal growth.

Responsibilities - What You Will Be Doing

Design, build, and manage the data processing pipelines and solutions used for training and testing of AI algorithms
Collaborate closely with perception and navigation R&D team members to implement machine learning algorithms and models tailored for high-accuracy offline execution, later to be used for automatic labeling and semi-supervised training
Profiling, analyzing, and benchmarking of models to identify areas for improvement in terms of efficiency, and accuracy emphasizing aspects of real-time performance on embedded platforms
Stay up to date with the latest advancements in deep learning foundation models and ML algorithms, data management solutions, and cloud computing advancements
Collaborate with the data management and cloud teams to guide and build a unified data management solution based on multi-sensors and our various data sources

Requirements:

Requirements - does this describe you?

BSc in Computer Science or Software Engineering or equivalent degree
Solid background in cloud-based workflows, acquaintance with existing data management and infrastructure frameworks, and understanding of tradeoffs and limitations of different solutions
Proficiency in Python programming, experience working with relevant deep learning and data processing libraries (Pytorch /TensorFlow, Pandas)
Proven experience with designing and building pipelines for Machine Learning projects based on the implementation of SOTA algorithms

Advantages – this will bring you to our front-row

 Expertise in one or more perception-related aspects of computer vision are a large advantage (SLAM, sensor fusion, intrinsic/extrinsic calibration, 3D reconstruction, etc.
Experience with C++ and ROS programming
"""

In [16]:
mobileye = """Which department will you join? 

The Cloud team of the Development Infrastructure (DI) department is responsible for Mobileye's Cloud infrastructure on AWS and Google Cloud. We develop mainly in Python, and some Golang. We are developing highly efficient systems for the company, running on up to 500K CPUs, processing hundreds of Petabytes of data monthly collected from all over the world. We also provide networking, security, governance, support, and guidance for all of Mobileye's diverse uses of AWS across tens of AWS accounts, including ML Training and AWS China. 

What will your job look like? 

Take part in the design, development, deployment, and maintenance of Mobileye's main cloud systems: Backend as well and python packages.
Hyper-scale batch processing framework, spinning up hundreds of thousands of parallel CPUs 
Systems to ingest, store and manage data from cameras and other sensors, of exceptional volumes - 100 PB and more. 
Deliver a High Quality product, fully developed by our team 
Work with highly experienced developers and architects 
Work tightly with AWS teams, bringing their services to the edge - S3, EC2, Spot, EBS, ECS, Batch, DynamoDB 
Build systems and tools using AWS Lambda, RDS, CloudFormation/Terraform and more 
See your product being used by hundreds of users, where every improvement, optimization and new feature is based on real requirements and is very significant to the development of autonomous vehicles and car safety systems. 

All you need is: 

B.Sc. Degree in Computer Science/Engineering 
At least 1-2 years of software development experience 
Good Team Player 
Excellent knowledge of Python 
Experience in Linux Environment 

Advantage skills and certifications: 

Experience with AWS services - an advantage 
Experience with Containers/Docker """


ai21_description = """
We are looking for a skilled Data Scientist to join our Core Data Science team at AI21. As a Data Scientist, you will own models in production that will serve effective and insightful decision-making processes, and will be used on a daily basis. Your value-driven mindset and strong business acumen will be critical to deliver impact. As part of a cross-company team, you will work closely with Marketing and Business Analytics Partners, Data Scientists and Data Engineers while developing end-to-end methodologies and models, such as LTV, financial forecasts and budget optimisations. These will contribute directly to AI21 Labs Business and Marketing success.

Role and Responsibilities:

Leverage both Data Science toolbox (ML, Causality, Experimentation, NLP, Time Series, etc.) and engineering skills to design, build, maintain, and improve multiple decision-making systems. Their adoption will determine their success.
Implement models in a scalable, robust, and maintainable manner using modern software development techniques.
Become an expert in data modeling and apply knowledge to our ETLs, complex data analyses and data enrichment.
Work closely with Analytics Partners, business stakeholders, Data Scientists and Data Engineers to foster a collaborative environment and great flow of information.

Requirements:

BSc in CS / Engineering / Statistics or equivalent (MSc is a plus). 
4+ years of experience as a Data Scientist in Marketing or Business departments.
Experience in owning end-to-end development of ML systems (ideation, prototyping, productionizing and evaluation), both engineering and algorithmic.
Proficiency in Python scientific packages (numpy, pandas, scikit-learn) and visualization tools (matplotlib, seaborn, bokeh).
Proficiency in SQL and/or Spark.
Strong knowledge in at least one of the following areas: System Design, Experimentation, Causality, Predictive Modeling or Optimisation.
Self motivated and curious, with a strong belief that data is essential for success

About Us:

AI21 Labs is a first mover generative AI company co-founded by AI luminaries and serial entrepreneurs Prof. Amnon Shashua (Hebrew U, Mobileye), Prof. Yoav Shoham (Stanford, Google), and Ori Goshen (8200, Crowdx) with the mission to empower businesses with state-of-the-art LLMs and AI applications.

Knowing that no single LLM can be the best at all tasks, AI21 has developed new types of hyper focused language models that each excel at a specific capability. In November 2023 AI21 Labs closed an oversubscribed Series C funding of $208 million at a valuation of $1.4 billion, which included investments from Google, Nvidia, Intel Capital, and Comcast Ventures. AI21 was among the first companies to bring generative AI to the masses and in addition to its enterprise solutions it also offers a popular application - Wordtune, used by millions of professionals.
"""
mobileye_keywords = """Cloud team, Development Infrastructure (DI) department, AWS, Google Cloud, Python, Golang, efficient systems, CPUs, Petabytes of data, networking, security, governance, ML Training, AWS China, design, deployment, maintenance, Backend, batch processing framework, data ingestion, data storage, AWS services (S3, EC2, Spot, EBS, ECS, Batch, DynamoDB), AWS Lambda, RDS, CloudFormation, Terraform, B.Sc. Degree in Computer Science/Engineering, software development, Team Player, Linux Environment, Containers, Docker.
'Cloud team', 'Development Infrastructure (DI) department', 'AWS', 'Google Cloud', 'Python', 'Golang', 'efficient systems', 'CPUs', 'Petabytes of data', 'networking', 'security', 'governance', 'ML Training', 'AWS China', 'design', 'deployment', 'maintenance', 'Backend', 'batch processing framework', 'data ingestion', 'data storage', 'AWS services (S3', 'EC2', 'Spot', 'EBS', 'ECS', 'Batch', 'DynamoDB)', 'AWS Lambda', 'RDS', 'CloudFormation', 'Terraform', 'B.Sc. Degree in Computer Science/Engineering', 'software development', 'Team Player', 'Linux Environment', 'Containers', 'Docker.'"""

checkpoint_keywords = "Cloud software developer, Backend development, Frontend development, Security hub, Cloud environments, CloudGuard, New technologies, Internal services, External services, Scaling, Tech leaders, Tools development, Computer Science, Python, Object-oriented design, Data structures, Algorithm design, Problem solving, Complexity analysis, Computer networking, Public cloud, AWS, Fast learner, Motivation, Time management, Interpersonal skills, Team player"

bluewhite_keywords = "'Data and ML Engineer', 'ML infrastructure', 'Data processing pipelines', 'AI algorithms', 'Machine learning-based perception', 'Navigation algorithms', 'Deep learning', 'Cloud computing', 'Python programming', 'Pytorch', 'TensorFlow', 'Pandas', 'SOTA algorithms', 'Computer vision', 'SLAM', 'Sensor fusion', '3D reconstruction', 'C++', 'ROS programming'"

tymly_keywords = "AI Agent, eCommerce, customer service, backend developer, server-side, pipelines, web applications, Back-end developer, Python, cloud-native software, AWS, cloud automation, Serverless, clean code, debugging, coding standards, developers, designers, new features, emerging technologies."


marke_keywords = "AI Engineer, Crawling, Synthetic Datasets, AI agents, Connect to external APIs, Create internal APIs, FineTune Large Language Models, HugginFace, Torch, NLP, LLMs, Python, AWS, GO, Ruby, Elixir, Erlang, Git, Communication, Interpersonal skills, Independent work, Team collaboration"
#todo fix adgusted bio not using keywords
org_bio = """Highly motivated individual with a keen interest in Machine Learning. Equipped with solid communication skills and a pas-
sion for problem-solving, I’m eager to contribute to a high-performing team in a fast-paced, technology-driven environment.
Dedicated to continuing leveraging my knowledge and skill-set by providing impactful solutions"
"""
BIO_REWRITING_SYS_PMT = f"""You are an expert in creating and editing cvs and resumes from your background in programming, nlp and data science i will give you a job description followed by keywords from that job description followed by a bio from my resume and i want you to adjust the bio according to the job description and keywords. only slightly change the bio and do not exceed the original bio length, and dont talk about experience talk about motivation and intrests  for example: job decription: {marketeam_description} , keywords: {marke_keywords} ,bio: {org_bio}. adjusted bio: 'Highly motivated individual with a keen interest in Machine Learning and NLP in particular, with experience in agentic
workflows. Equipped with solid communication skills and a passion for problem-solving. I’m eager to contribute to a high-
performing team in a fast-paced, technology-driven environment. Dedicated to continuing leveraging my knowledge and
skill-set by providing impactful solutions.'"""

print(ask_gpt(BIO_REWRITING_SYS_PMT, f"Job decription: {mobileye}, keywords: {mobileye_keywords}, resume bio: {org_bio}"))

checkpoint : {Azure, AWS, GCP, object-oriented design, python, c++, }


marketeam :

my keywords {'nlp', 'Ai agents', 'LLM APIs', 'Finetuning', 'HugginFace', 'Torch', 'LLMs', 'Git', 'AWS', 'GO', 'Ruby', 'Elixir', 'Erlang', 'Python'}

bio: "Highly motivated individual with a keen interest in Machine Learning and GenAI in particular, with experience in agentic workflows. Equipped with solid communication skills and a passion for problem-solving. I'm eager to contribute to a high-performing team in a fast-paced, technology-driven environment. Dedicated to continue leveraging and growing my skill-set building impactful tools. "


keys = {'SOLID principles', 'build tools.', 'scalability', 'Scala', 'team player', 'developer tooling', 'Object-Oriented Programming (OOP)', 'software engineering best practices', 'debugging', 'backend engineering', 'development environments', 'Kubernetes', 'microservice architecture', 'code reviews', 'Docker', 'server-side technologies', 'DevEx-related topics', 'Bazel', 'CI/CD', 'DevPortal', 'testing', 'industry trends'}



In [35]:

#todo dosent work
# res = rank_keywords(keywords, wix_description)
# print(res)
print("all of the keywords ", keywords)
print(ask_gpt(f"split the following keywords into two catagories: 1- skills-qualifications, 2- attributes-abstract-concepts. each of the keywords can only be in one of the catagories return your response like this: 1-docker, oop, ruby, 2-team player, code reviews, motivated", str(keywords)))

all of the keywords  ['Cloud software developer', 'Backend development', 'Frontend development', 'Security hub', 'Cloud environments', 'CloudGuard', 'New technologies', 'Internal services', 'External services', 'Scaling', 'Tech leaders', 'Tools development', 'Computer Science', 'Server-side languages', 'Python', 'Object-oriented design', 'Data structures', 'Algorithm design', 'Problem solving', 'Complexity analysis', 'Computer networking', 'Troubleshooting', 'Public cloud', 'AWS', 'Fast learner', 'Motivation', 'Time management', 'Interpersonal skills', 'Team player.']
1- CloudGuard, Server-side languages, Python, Object-oriented design, Data structures, Algorithm design, Problem solving, Complexity analysis, Computer networking, Troubleshooting, Public cloud, AWS  
2- New technologies, Internal services, External services, Scaling, Tech leaders, Tools development, Computer Science, Fast learner, Motivation, Time management, Interpersonal skills, Team player


In [20]:
# best_keywords= ['SOLID principles',  'Object-Oriented Programming', 'CI/CD',  'Docker', 'Kubernetes', 'Scala']



